构建自定义层

首先构造不带参数的层

类似于之前定义块的方法

在下面这个定义的块中，减去了输入数据的平均值，可以视为中心化或者零均值化

In [ ]:
import torch
import torch.nn.functional as F
from torch import nn

class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, X):
        return X - X.mean()

通过提供实例进行尝试

In [2]:
layer = CenteredLayer()
layer(torch.FloatTensor([1, 2, 3, 4, 5]))

tensor([-2., -1.,  0.,  1.,  2.])

定义完成后就可以将其作为一个组件合并到更为复杂的模型中

In [3]:
net = nn.Sequential(nn.Linear(8, 128), CenteredLayer())

作为健全性检查，可以后续检查均值是否为0

In [4]:
Y = net(torch.rand(4, 8))
Y.mean()

tensor(6.9849e-09, grad_fn=<MeanBackward0>)

接着就可以尝试构造带参数的层

这些参数可以通过训练进行调整。 
我们可以使用内置函数来创建参数，这些函数提供一些基本的管理功能。 
比如管理访问、初始化、共享、保存和加载模型参数。 这样做的好处之一是：我们不需要为每个自定义层编写自定义的序列化程序。


现在，让我们实现自定义版本的全连接层。
回想一下，该层需要两个参数，一个用于表示权重，另一个用于表示偏置项。 在此实现中，我们使用修正线性单元作为激活函数。 
该层需要输入参数：in_units和units，分别表示输入数和输出数。（输入特征维度和输出特征维度）

通过将张量包装为 nn.Parameter，PyTorch 会：
    自动将其注册为模块的参数
    在反向传播中计算其梯度
    在调用 model.parameters() 时包含它
    在保存/加载模型时正确处理它

In [5]:
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        # nn.Parameter() 将这个张量包装成一个参数对象，这是 PyTorch 中表示可学习参数的特殊类型
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units,))

    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)

接下来实例化并访问参数

In [6]:
linear = MyLinear(5, 3)
linear.weight

Parameter containing:
tensor([[-0.4994, -1.5832, -0.4318],
        [ 0.3443,  0.4894, -0.2987],
        [-1.1714, -0.1529,  0.1242],
        [ 0.0248,  2.0992,  0.6486],
        [-1.1016, -0.4975,  0.1530]], requires_grad=True)

因为使用Parameter进行了参数化，因此可以直接用自定义层执行前向传播计算

In [7]:
linear(torch.rand(2, 5))

tensor([[0.0000, 0.0000, 0.2352],
        [0.0000, 0.3982, 0.7668]])

完全可以用自定义层来构建模型

In [8]:
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))

tensor([[0.5113],
        [0.0000]])